**Set up CUDA Quantum Environment**

In [1]:
"""
#The following commands are used to set up the environment in Colab
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.0-1_all.deb
!dpkg -i cuda-keyring_1.0-1_all.deb
!apt-get update
!apt-get -y install libcublas-11-8 libcusolver-11-8 cuda-cudart-11-8
%pip install cuda-quantum==0.6.0
"""

'\n!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.0-1_all.deb\n!dpkg -i cuda-keyring_1.0-1_all.deb\n!apt-get update\n!apt-get -y install libcublas-11-8 libcusolver-11-8 cuda-cudart-11-8\n%pip install cuda-quantum==0.6.0\n'

# CUDA Quantum 101
    Important Links
    
    * Perlmutter Specific Instructions
        https://github.com/poojarao8/nersc-quantum-day/blob/master/PerlmutterInstructions.md
    * Installation (Docker recommended)
        https://nvidia.github.io/cuda-quantum/latest/install.html
    * Documentation
        https://nvidia.github.io/cuda-quantum/latest/index.html
    * CUDA Quantum Repo
        https://github.com/NVIDIA/cuda-quantum
    * Scaling Applications
        https://nvidia.github.io/cuda-quantum/latest/examples/python/tutorials/multi_gpu_workflows.html


    Outline

    1. What is CUDA Quantum?
    2. CUDA Quantum Kernels
    3. CUDA Quantum Primitives
        3.1 cudaq.sample()
        3.2 cudaq.spin_op()
        3.3 cudaq.observe()
    4. Parameterized circuits
    5. Noise-modeling
    6. Scaling

### 1. CUDA Quantum

    - NVIDIA’s open-source platform for hybrid quantum-classical computing

    - Built for high-performance, scalability, and ease-of-use

    - As all valuable quantum applications of the future will be hybrid, CUDA Quantum enables users to develop performant hybrid applications that can easily scale to supercomputing scale systems like NERSC’s Perlmutter.

<div style="display:flex;justify-content:center;">
    <img src="figs/doe_excerpts.png" alt="Image Title" width="600">
</div>


#### 2. CUDA Quantum Kernel

In [2]:
# Import the CUDA Quantum module
import cudaq

In [3]:
# We begin by defining the `Kernel` that we will construct our
# program with.
kernel = cudaq.make_kernel()

In [4]:
# Next, we can allocate qubits to the kernel via `qalloc(qubit_count)`.
# An empty call to `qalloc` will return a single qubit.
qubit = kernel.qalloc()

In [5]:
# Now we can begin adding instructions to apply to this qubit!
# Here we'll just add every non-parameterized
# single qubit gates that are supported by CUDA Quantum.
kernel.h(qubit)
kernel.x(qubit)
kernel.y(qubit)
kernel.z(qubit)
kernel.t(qubit)
kernel.s(qubit)

In [6]:
# Next, we add a measurement to the kernel so that we can sample
# the measurement results on our simulator!
kernel.mz(qubit)

In [7]:
# Other methods and attributes available to the kernel object
dir(kernel)
#help(kernel.tdg)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'adjoint',
 'apply_call',
 'argument_count',
 'arguments',
 'c_if',
 'ch',
 'control',
 'cr1',
 'crx',
 'cry',
 'crz',
 'cs',
 'cswap',
 'ct',
 'cx',
 'cy',
 'cz',
 'exp_pauli',
 'fermionic_swap',
 'for_loop',
 'givens_rotation',
 'h',
 'mx',
 'my',
 'mz',
 'name',
 'qalloc',
 'r1',
 'reset',
 'rx',
 'ry',
 'rz',
 's',
 'sdg',
 'swap',
 't',
 'tdg',
 'to_quake',
 'x',
 'y',
 'z']

###     3. Algorithmic primitives

  Algorithmic primitives are common programming patterns that have
  been implemented in the CUDA Quantum library.

    3.1 cudaq.sample()
    3.2 cudaq.observe()
    3.3 cudaq.spin_op()

#### 3.1. cudaq.sample()

      The sample() function performs multiple measurements of the
      circuit(1000 shots by default) and returns a dictionary of the
      measurement outcomes along with their respective counts.

In [8]:
# Finally, we can execute this kernel on the state vector simulator
# by calling `cudaq.sample`. This will execute the provided kernel
# `shots_count` number of times and return the sampled distribution
# as a `cudaq.SampleResult` dictionary.
sample_result = cudaq.sample(kernel)

# Now let's take a look at the `SampleResult` we've gotten back!
print(sample_result)  # or result.dump()

{ 0:515 1:485 }



    Putting it all together!

In [9]:
import cudaq

kernel = cudaq.make_kernel()
qubit = kernel.qalloc(2)

kernel.h(qubit)
kernel.x(qubit)
kernel.y(qubit)
kernel.z(qubit)
kernel.t(qubit)
kernel.s(qubit)

kernel.mz(qubit)

# 1000 is the default
sample_result = cudaq.sample(kernel, shots_count=2000)

print(sample_result)  # or sample_result.dump()

{ 00:519 10:526 01:457 11:498 }



In [10]:
# Extracting data from sample

print(f"most probable = {sample_result.most_probable()}")
print(f"expectation_value = {sample_result.expectation_z()}")
print(f"count = {sample_result.count('1')}")
print(f"probability = {sample_result.probability('1')}")


most probable = 10
expectation_value = 0.016999999999999987
count = 0
probability = 0.0


/tmp/ipykernel_14018/221317482.py:4: DeprecationWarning: expectation_z() is deprecated, use expectation() with the same argument structure.
  print(f"expectation_value = {sample_result.expectation_z()}")


In [11]:
# clear results, result should now be empty
sample_result.clear()
print(sample_result)

{ }



####  3.2. cudaq.spin_op()

     
    The spin_op represents a sum of Pauli tensor products.
    
    - Typical algebraic operations can be used to compose larger,
    more complex Pauli tensor products and their sums.

Let's take the Hamitonian H such that, H  = $Z_0 \otimes I_1 + I_0 \otimes X_1 + Y_0 \otimes I_1 + Y_0 \otimes Y_1$.

In [12]:
# Importing the spin_op
from cudaq import spin

# the obseravle
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0)*spin.y(1)

# add some more terms
for i in range(2):
  hamiltonian += -2.0*spin.z(i)*spin.z(i+1)

print(hamiltonian)
print(hamiltonian.to_matrix())
print(hamiltonian.to_sparse_matrix())


[-2+0j] IZZ
[1+0j] ZII
[1+0j] YII
[1+0j] IXI
[1+0j] YYI
[-2+0j] ZZI

(-3,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0) (-1,0)  (0,0)
 (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0) (-1,0)
 (1,0)  (0,0)  (5,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0)
 (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (0,1)
(0,-1)  (0,0)  (1,0)  (0,0) (-1,0)  (0,0)  (1,0)  (0,0)
 (0,0) (0,-1)  (0,0)  (1,0)  (0,0)  (3,0)  (0,0)  (1,0)
(-1,0)  (0,0) (0,-1)  (0,0)  (1,0)  (0,0) (-1,0)  (0,0)
 (0,0) (-1,0)  (0,0) (0,-1)  (0,0)  (1,0)  (0,0) (-5,0)

([(-3+0j), (1+0j), 1j, (-1+0j), (1+0j), (1+0j), 1j, (-1+0j), (1+0j), (5+0j), (1+0j), 1j, (1+0j), (1+0j), (1+0j), 1j, -1j, (1+0j), (-1+0j), (1+0j), -1j, (1+0j), (3+0j), (1+0j), (-1+0j), -1j, (1+0j), (-1+0j), (-1+0j), -1j, (1+0j), (-5+0j)], [0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7], [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7])


In [13]:
dir(hamiltonian)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'distribute_terms',
 'dump',
 'for_each_pauli',
 'for_each_term',
 'from_word',
 'get_coefficient',
 'get_qubit_count',
 'get_raw_data',
 'get_term_count',
 'is_identity',
 'random',
 'serialize',
 'to_matrix',
 'to_sparse_matrix',
 'to_string']

#### 3.3. cudaq.observe()

Compute the expected value of the observable, i.e., $\bra{\psi}H\ket{\psi}$, where $H$ is a cudaq spin_op.

In [14]:
# Importing the spin_op
from cudaq import spin

# the obseravle
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0)*spin.y(1)

# First we need to construct a cuda quantum kernel
kernel = cudaq.make_kernel()
qreg = kernel.qalloc(2)
kernel.x(qreg[0])

# The cudaq.observe() takes the quantum circuit and the observable as input params
observe_result = cudaq.observe(kernel, hamiltonian, shots_count=1000)

In [15]:
print(observe_result)
observe_result.expectation_z()

/tmp/ipykernel_14018/295237787.py:2: DeprecationWarning: expectation_z() is deprecated, use expectation() with the same argument structure.
  observe_result.expectation_z()


-1.068

In [16]:
# For a complete list of attributes
# dir (observe_result)

### 4. Parameterized circuits

In [17]:
import cudaq
from cudaq import spin

# the obserable
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) \
            - 2.1433 * spin.y(0) * spin.y(1) + 0.21829 * spin.z(0) \
            - 6.125 * spin.z(1)

# parameterized cudaq kernel, the parameter is of type float
kernel, theta = cudaq.make_kernel(float)
q = kernel.qalloc(2)
kernel.x(q[0])
kernel.ry(theta, q[1])
kernel.cx(q[1], q[0])

# observe() takes the kernel, the observable and the kernel paramter(s)
# as args
observe_result = cudaq.observe(kernel, hamiltonian, .59)
observe_result.expectation_z()

/tmp/ipykernel_14018/1216256030.py:19: DeprecationWarning: expectation_z() is deprecated, use expectation() with the same argument structure.
  observe_result.expectation_z()


-1.7487943680728968

### 4. Noise modeling

    Noise can be represnted mathematically using the Kraus operators.
    
\begin{equation*}
\rho \mapsto {\cal{N}}(\rho) = \sum_{j} K_j \rho K_j^{\dag}
\end{equation*}

    with the condition that
    
\begin{equation*}
\sum_{j} K_j K_j^{\dag} = \mathbb{I}.
\end{equation*}

    A single-qubit bit-flip error can be expressed as:

\begin{equation*}
    \rho = (1-p) \rho + p X\rho X
\end{equation*}
    with p in [0,1].


In [18]:
import cudaq

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# We will begin by defining an empty noise model that we will add
# these decoherence channels to.
noise = cudaq.NoiseModel()

# Bit flip channel with `1.0` probability of the qubit flipping 180 degrees.
bit_flip = cudaq.BitFlipChannel(1.0)
# We will apply this channel to any X gate on the qubit, giving each X-gate
# a probability of `1.0` of undergoing an extra X-gate.
noise.add_channel('x', [0], bit_flip)

# construct a circuit
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Apply an X-gate to the qubit.
# It will remain in the |1> state with a probability of `1 - p = 0.0`.
kernel.x(qubit)
kernel.mz(qubit)

# noisy simulation
noisy_result = cudaq.sample(kernel, noise_model=noise)
print(noisy_result)

# noiseless simulation
noiseless_result = cudaq.sample(kernel)
print(noiseless_result)

{ 0:1000 }

{ 1:1000 }



 Custom Noise Model

     Here, we demonstrate a custom noise model with the same Kraus operators as in the ampltiude damping channel, but following the same template we can build other noise models such as the Pauli noise model.

In [19]:
import cudaq
import numpy as np

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# We will begin by defining an empty noise model that we will add
# our Kraus Channel to.
noise = cudaq.NoiseModel()

# We will define our Kraus Operators within functions, as to
# allow for easy control over the noise probability.
def kraus_operators(probability):
    """See Nielsen, Chuang Chapter 8.3.5 for definition source."""
    kraus_0 = np.array([[1, 0], [0, np.sqrt(1 - probability)]],
                       dtype=np.complex128)
    kraus_1 = np.array([[0, 0], [np.sqrt(probability), 0]], dtype=np.complex128)
    return [kraus_0, kraus_1]


# Manually defined amplitude damping channel with `1.0` probability
# of the qubit decaying to the ground state.
amplitude_damping = cudaq.KrausChannel(kraus_operators(1.0))
# We will apply this channel to any Hadamard gate on the qubit.
noise.add_channel('h', [0], amplitude_damping)

# construct a simple kernel
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()
kernel.h(qubit)
kernel.mz(qubit)

# noisy
noisy_result = cudaq.sample(kernel, noise_model=noise)
print(noisy_result)

# noiseless
noiseless_result = cudaq.sample(kernel)
print(noiseless_result)

{ 0:1000 }

{ 0:517 1:483 }



#### Single-gpu speedup

In [20]:
import cudaq

#cudaq.set_target("default") # cpu
cudaq.set_target("nvidia") # single gpu acceleration

def ghz_state(N):
    kernel = cudaq.make_kernel()
    q = kernel.qalloc(N)
    kernel.h(q[0])
    for i in range(N - 1):
      kernel.cx(q[i], q[i + 1])

    kernel.mz(q)
    return kernel

n = 30
print("Preparing GHZ state for", n, "qubits.")
kernel = ghz_state(n)
counts = cudaq.sample(kernel)
print(counts)

Preparing GHZ state for 30 qubits.
{ 000000000000000000000000000000:525 111111111111111111111111111111:475 }



### Scaling applications in CUDA Quantum
Main reference: https://nvidia.github.io/cuda-quantum/latest/examples/python/tutorials/multi_gpu_workflows.html

Targets

    - A combination of quantum circuit simulators and hardware.
    - Allows you to switch between QPUs, CPUs and GPUs.
    - The default target provides a state vector simulator based on the CPU-only, OpenMP threaded Q++ library.


Available Targets¶

        default: The default qpp based CPU backend which is multithreaded to maximise the usage of available cores on your system.

        nvidia: GPU based backend which accelerates quantum circuit simulation on NVIDIA GPUs powered by cuQuantum.

        nvidia-mqpu: Enables users to program workflows utilizing multiple quantum processors enabled today by GPU emulation.

        nvidia-mgpu: Allows for scaling circuit simulation beyond what is feasible with any QPU today.

        density-matrix-cpu: Noisy simulations via density matrix calculations. CPU only for now with GPU support coming soon.

In [21]:
# Print all the availble targets for your system
import cudaq

targets = cudaq.get_targets()

for target in targets:
    print(target)

Target nvidia-mgpu
	simulator=nvidia_mgpu
	platform=default
	description=

Target tensornet
	simulator=tensornet
	platform=default
	description=

Target tensornet-mps
	simulator=tensornet_mps
	platform=default
	description=

Target density-matrix-cpu
	simulator=dm
	platform=default
	description=The Density Matrix CPU Target provides a simulated QPU via OpenMP-enabled, CPU-only density matrix emulation.

Target iqm
	simulator=qpp
	platform=default
	description=

Target nvidia-mqpu-fp64
	simulator=custatevec_fp64
	platform=mqpu
	description=The NVIDIA MQPU FP64 Target provides a simulated QPU for every available CUDA GPU on the underlying system. Each QPU is simulated via cuStateVec FP64.

Target nvidia-fp64
	simulator=custatevec_fp64
	platform=default
	description=The NVIDIA FP64 Target provides a simulated QPU via single-GPU cuStateVec integration on FP64 types.

Target nvidia
	simulator=custatevec_fp32
	platform=default
	description=The NVIDIA Target provides a simulated QPU via singl

    Some  ways to scale your application:
  
    1. Increasing the number of qubits (weak scaling)
    
            - mgpu backend
    
    2. Distributing the circuit execution (strong scaling)
            2.1 asynchronous sampling
            2.2 Hamiltonian batching
            2.3 Parameter batching

            - mqpu backend
            - Each gpu acts as a virtual qpu

         As a rule of thumb, we can parallelize over any of the input parameters to `cudaq.sample()` or `cudaq.observe()` - kernel, hamiltonian, kernel parameters, etc.

### Multiple NVIDIA GPUs for the mgpu backend

    - The increase in qubit count leads to an exponential increase in the size of the statevector.
    
    - The nvidia-mgpu target allows for scaling the qubit count by pooling memory from GPUs across multiple nodes.

    - Execution on the nvidia-mgpu backed is enabled via `srun` on Perlmutter.

    - To test this, run the GHZ state prep example https://github.com/poojarao8/nersc-quantum-day/blob/master/ghz.py using the instructions from here https://github.com/poojarao8/nersc-quantum-day/blob/master/PerlmutterInstructions.md.  

                            
                            GHZ state prep on Perlmutter
<div style="display:flex;justify-content:center;">
    <img src="figs/qubit_scaling.png" alt="Image Title" width="200">
</div>


### Asynchronous sampling via mqpu backend

In [22]:
import cudaq

cudaq.set_target("nvidia-mqpu")
target = cudaq.get_target()
num_qpus = target.num_qpus()
print("Number of QPUs:", num_qpus)

kernel = cudaq.make_kernel()
qubits = kernel.qalloc(2)
kernel.h(qubits[0])
kernel.cx(qubits[0], qubits[1])
kernel.mz(qubits)

futures = []
for i in range(num_qpus):
  futures.append(cudaq.sample_async(kernel, qpu_id=i))


for count in futures:
    print(count.get())


Number of QPUs: 2
{ 00:472 11:528 }

{ 00:487 11:513 }



      Asynchronous expectation value computation

In [23]:
import cudaq
from cudaq import spin

kernel = cudaq.make_kernel()
qubit = kernel.qalloc()
kernel.x(qubit)

# Measuring in the Z-basis.
hamiltonian = spin.z(0)

# Call `cudaq.observe()` at the specified number of shots.
future = cudaq.observe_async(kernel=kernel,
                            spin_operator=hamiltonian,
                            qpu_id=0,
                            shots_count=2000)
observe_result = future.get()
got_expectation = observe_result.expectation_z()

/tmp/ipykernel_14018/2462415777.py:17: DeprecationWarning: expectation_z() is deprecated, use expectation() with the same argument structure.
  got_expectation = observe_result.expectation_z()


                    Hamiltonian term distribution over multiple QPUs
<div style="display:flex;justify-content:center;">
    <img src="figs/hamiltonian_batch.png" alt="Image Title" width="500">
</div>


In [24]:
import cudaq

cudaq.set_target("nvidia-mqpu")

qubit_count = 15
term_count = 100

kernel = cudaq.make_kernel()
qubits = kernel.qalloc(qubit_count)
kernel.h(qubits[0])

for i in range(1, qubit_count):
    kernel.cx(qubits[0], qubits[i])

# We create a random hamiltonian with several terms
hamiltonian = cudaq.SpinOperator.random(qubit_count, term_count)

# The observe calls allows us to calculate the expectation value of the Hamiltonian,
# batches the terms, and distributes them over the multiple QPU's/GPUs.
# expectation = cudaq.observe(kernel, hamiltonian)  # Single node, single GPU.

expectation = cudaq.observe(kernel, hamiltonian,
                        execution=cudaq.parallel.thread)  # Single node, multi-GPU.

# expectation = cudaq.observe(kernel, hamiltonian,
#                       execution= cudaq.parallel.mpi) # Multi-node, multi-GPU.

    More on workflows enabled by the use of multiple gpus:
https://nvidia.github.io/cuda-quantum/latest/examples/python/tutorials/multi_gpu_workflows.html

### Other useful things

    Adjoing of a kernel

In [25]:
 import cudaq

# Create a kernel and do some operations
other_kernel = cudaq.make_kernel()
other_qubit = other_kernel.qalloc()
other_kernel.x(other_qubit)

# Create a kernel, which'll be the adjoint of other_kernel
kernel = cudaq.make_kernel()
kernel.adjoint(other_kernel)

    Conditional Measurement

In [26]:
 # The conditional measurement functionality of `cudaq.kernel`
import cudaq

kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

def then_function():
    kernel.x(qubit)

kernel.x(qubit)

# Measure the qubit.
measurement_ = kernel.mz(qubit)
# Apply `then_function` to the `kernel` if
# the qubit was measured in the 1-state.
kernel.c_if(measurement_, then_function)

# Measure the qubit again.
result = cudaq.sample(kernel, shots_count=30)
print(result)


{ 
  __global__ : { 0:30 }
   auto_register_0 : { 1:30 }
}



### Variational Algorithms

    Variational algorithms in CUDA Quantum typically leverage the `cudaq.observe(...)` function in tandem with the `cudaq.optimizer`.

    One can choose an optimization strategy provided as specific sub-types of the `cudaq.optimizers`.

In [27]:
# Import the necessary modules
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
kernel, theta = cudaq.make_kernel(list)
qreg = kernel.qalloc(2)
kernel.x(qreg[0])
kernel.ry(theta[0], qreg[1])


# Observable
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.SPSA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 2

def cost_function(x):
    # cudaq.observe() produces the expected value of a specified observable wrt a given parameterized ansatz at given params.
    # This value is the cost function wrt which we are optimizing.
    observeResult = cudaq.observe(kernel, hamiltonian, x)
    print (observeResult.expectation_z(), x)
    return observeResult.expectation_z()

# Carry out the optimization
opt_value, opt_theta = optimizer.optimize(dimensions=1, function=cost_function)

-0.7044796496629715 [0.3]
-1.2955199629068375 [-0.3]
-1.1567716915160418 [-0.1574212903296091]
-0.8780100231524557 [0.1222946556014406]
-1.423347532749176 [-0.4371372362606588]
-1.2571947555989025 [-0.26011805461908494]
-1.2571947555989025 [-0.26011805461908494]


/tmp/ipykernel_14018/166767970.py:25: DeprecationWarning: expectation_z() is deprecated, use expectation() with the same argument structure.
  print (observeResult.expectation_z(), x)
/tmp/ipykernel_14018/166767970.py:26: DeprecationWarning: expectation_z() is deprecated, use expectation() with the same argument structure.
  return observeResult.expectation_z()


In [28]:
# Other optimizers and attributes available to the `cudaq.optimizers`
dir(cudaq.optimizers)

['Adam',
 'COBYLA',
 'GradientDescent',
 'LBFGS',
 'NelderMead',
 'SGD',
 'SPSA',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'optimizer']

    VQE wrapper

In [29]:
 # Import the necessary modules
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
kernel, theta = cudaq.make_kernel(list)
qreg = kernel.qalloc(2)
kernel.x(qreg[0])
kernel.ry(theta[0], qreg[1])

# Hamiltonian operator
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.COBYLA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 5

# Carry out the optimization
opt_value, opt_theta = cudaq.vqe(kernel=kernel,
                        spin_operator=hamiltonian,
                        optimizer=optimizer,
                        parameter_count=1)

print(f"\nminimized <H> = {round(opt_value,16)}")
print(f"optimal theta = {round(opt_theta[0],16)}")


minimized <H> = -1.9999997019767757
optimal theta = -1.5707963267948963
